In [86]:
#visualization part
#import matplotlib.pyplot as plt
#import seaborn as sns


#basic libs
import pandas as pd
import numpy as np
import random as rnd

from sklearn.model_selection import cross_val_score

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



train_=pd.read_csv('../train_allcols.csv')
validate_=pd.read_csv('../validate_allcols.csv')
#test=pd.read_csv('../testwDSM.csv')

train_.shape, validate_.shape, #test.shape

((301619, 62), (148560, 62))

In [87]:
#train.describe()
train = train_#.sample(40000)
validate = validate_#.sample(10000)
train.shape, #validate_.shape, validate.head(2)

((301619, 62),)

In [88]:
train = train.query('SUB2 != 1')
validate = validate.query('SUB2 != 1')

train = train[train.SUB2.isin([1,2,3,4,5,7,10])]
validate = validate[validate.SUB2.isin([1,2,3,4,5,7,10])]
#train = train[train.SUB1.isin([2,3,4,5,7,10])]
#validate = validate[validate.SUB1.isin([2,3,4,5,7,10])]

print train.shape
print train['SUB2'].value_counts()

(144256, 62)
4     51418
2     42014
3     23968
7     11755
10    10936
5      4165
Name: SUB2, dtype: int64


In [89]:
# for sub2



X_train = train.drop(['SUB2'], axis=1)
Y_train = train["SUB2"]

from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import f_classif,chi2
#Selector_f = SelectPercentile(f_classif, percentile=25)
Selector_f = SelectKBest(f_classif, k=10)
Selector_f.fit(X_train,Y_train)

zipped = zip(X_train.columns.tolist(),Selector_f.scores_)
ans = sorted(zipped, key=lambda x: x[1])
for n,s in ans:    
    if 'FLG' in n: 
        pass
    else:
        print 'F-score: %3.2ft for feature %s' % (s,n)
                
#print X_train.columns.tolist()

F-score: 14.17t for feature DAYWAIT
F-score: 17.34t for feature PREG
F-score: 21.70t for feature VET
F-score: 22.35t for feature ARRESTS
F-score: 66.57t for feature DETNLF
F-score: 81.89t for feature CBSA10
F-score: 87.31t for feature ETHNIC
F-score: 107.07t for feature PRIMINC
F-score: 122.83t for feature PRIMPAY
F-score: 139.18t for feature SUB3
F-score: 147.93t for feature RACE
F-score: 153.74t for feature REGION
F-score: 164.78t for feature LIVARAG
F-score: 231.84t for feature ROUTE3
F-score: 254.47t for feature STFIPS
F-score: 256.69t for feature PSYPROB
F-score: 259.85t for feature FRSTUSE1
F-score: 260.71t for feature GENDER
F-score: 269.05t for feature MARSTAT
F-score: 271.85t for feature EMPLOY
F-score: 304.52t for feature YEAR
F-score: 309.23t for feature CASEID
F-score: 386.59t for feature DIVISION
F-score: 422.86t for feature SERVSETA
F-score: 436.77t for feature FRSTUSE3
F-score: 442.57t for feature FREQ2
F-score: 471.46t for feature HLTHINS
F-score: 477.41t for feature DE

In [90]:
# get the sorted feature list
import sys
for n, s in ans:
    
    if 'FLG' in n: 
        pass
    else:
        sys.stdout.write('\'%s\',' % (n))


'DAYWAIT','PREG','VET','ARRESTS','DETNLF','CBSA10','ETHNIC','PRIMINC','PRIMPAY','SUB3','RACE','REGION','LIVARAG','ROUTE3','STFIPS','PSYPROB','FRSTUSE1','GENDER','MARSTAT','EMPLOY','YEAR','CASEID','DIVISION','SERVSETA','FRSTUSE3','FREQ2','HLTHINS','DETCRIM','EDUC','METHUSE','FREQ1','NOPRIOR','FREQ3','NUMSUBS','PSOURCE','SUB1','ROUTE1','DSMCRIT','AGE','IDU','ALCDRUG','ROUTE2','FRSTUSE2',

In [91]:
#train = train.query('SUB1 <= 10').query('SUB2 <= 10')
#validate = validate.query('SUB1 <= 10').query('SUB2 <= 10')

#drop_list = ['SUB2',  'ROUTE2', 'FREQ2', 'FRSTUSE2', 'SUB3', 'ROUTE3', 'FREQ3', 'FRSTUSE3', 'NUMSUBS'             ]


retain_list = ['EMPLOY','GENDER','FREQ1','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'NOPRIOR','DSMCRIT','ROUTE1','SUB1','AGE','IDU',
               #'YEAR','REGION','DIVISION',
               ]#'SUB3','ROUTE3','FREQ3','FRSTUSE3','FREQ2','FRSTUSE2']
X_train = train[retain_list]
Y_train = train["SUB2"]
X_validate = validate[retain_list]
Y_validate = validate["SUB2"]
#X_test  = test.drop(drop_list, axis=1)
X_train.shape, X_validate.shape, #X_test.shape



((144256, 14), (70664, 14))

In [92]:
retain_list = ['EMPLOY','GENDER','FREQ1','YEAR','EDUC','PSYPROB','PSOURCE','SERVSETA','DETCRIM',
               'REGION','NOPRIOR','DIVISION','DSMCRIT','ROUTE1','SUB1','AGE','IDU',
               ]
list1 = [u'AGE', u'GENDER', u'RACE', u'ETHNIC', u'MARSTAT', u'EDUC', u'EMPLOY', u'DETNLF', u'PREG',
         u'VET', u'LIVARAG', u'ARRESTS', u'SERVSETA', u'PSOURCE', u'DETCRIM', u'NOPRIOR', u'SUB1', u'ROUTE1', 
         u'FREQ1', u'FRSTUSE1', u'IDU', u'DSMCRIT', u'PSYPROB']

common = set(retain_list) & set(list1)
print list2 
print [item for item in retain_list if item not in common]

set(['PSOURCE', 'DSMCRIT', 'SUB1', 'GENDER', 'AGE', 'IDU', 'PSYPROB', 'EMPLOY', 'FREQ1', 'DETCRIM', 'ROUTE1', 'EDUC', 'NOPRIOR', 'SERVSETA'])
['YEAR', 'REGION', 'DIVISION']


In [93]:
print X_train.columns.tolist()

['EMPLOY', 'GENDER', 'FREQ1', 'EDUC', 'PSYPROB', 'PSOURCE', 'SERVSETA', 'DETCRIM', 'NOPRIOR', 'DSMCRIT', 'ROUTE1', 'SUB1', 'AGE', 'IDU']


In [94]:
#one hot
from sklearn import preprocessing

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(X_train)

# 3. Transform
onehotlabels = enc.transform(X_train).toarray()
#onehotlabels.shape
X_train = onehotlabels

onehotlabels = enc.transform(X_validate).toarray()
X_validate = onehotlabels

X_train.shape, X_validate.shape

((144256, 103), (70664, 103))

In [95]:
# Logistic Regression
logreg = LogisticRegression(C=1)
logreg.fit(X_train, Y_train)
#Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_validate, Y_validate) * 100, 2)
print acc_log


58.02


In [97]:
# Perceptron

perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
#Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_validate, Y_validate) * 100, 2)
print acc_perceptron


51.0


In [98]:
# Stochastic Gradient Descent

sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


56.22


In [99]:
# Random Forest (slow)

random_forest = RandomForestClassifier(n_estimators=100, max_depth=20)
random_forest.fit(X_train, Y_train)
#Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_validate, Y_validate) * 100, 2)
print acc_random_forest

#print cross_val_score(random_forest, X_validate, Y_validate)

59.21


In [100]:
# Stochastic Gradient Descent
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
#Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_validate, Y_validate) * 100, 2)
print acc_sgd


56.4


In [101]:
# Linear SVC
linear_svc = LinearSVC(C=1)
linear_svc.fit(X_train, Y_train)
#Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_validate, Y_validate) * 100, 2)
print acc_linear_svc


57.64


In [102]:
import xgboost as xgb
xgb_boost = xgb.XGBClassifier(seed=1850, n_jobs=-1)
xgb_boost.fit(X_train, Y_train) 
acc_xgb = round(xgb_boost.score(X_validate, Y_validate) * 100, 2)
print acc_xgb

57.48


In [103]:
print 'predict-top6-sub2-allcols-sample10000'
models = pd.DataFrame({
    'Model': [ 'Logistic Regression', 
              'Random Forest',  'Perceptron', 
              'Stochastic Gradient Decent', 'Linear SVC', 
               'XGBoost'],
    'Validate Set Score': [acc_log, 
              acc_random_forest,  acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_xgb]
    })
print models.sort_values(by='Validate Set Score', ascending=False)


predict-top6-sub2-allcols-sample10000
                        Model  Validate Set Score
1               Random Forest               59.21
0         Logistic Regression               58.02
4                  Linear SVC               57.64
5                     XGBoost               57.48
3  Stochastic Gradient Decent               56.40
2                  Perceptron               51.00
